In [1]:
# ======================================================================
# ⚙️ [연구 3단계] 토픽 모델링 및 '올바른' 키워드 추출
# ======================================================================
# 목표: 정제된 리뷰 데이터를 바탕으로 고객의 주요 관심 주제(Topic)를 분류하고,
#      각 주제를 대표하는 '의미 있는' 핵심 키워드를 계산하여 저장합니다.
# ======================================================================

# 1. 라이브러리 임포트 및 데이터 로드
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from ast import literal_eval
import os
import torch
from collections import Counter

try:
    df = pd.read_csv('../data/processed_reviews.csv')
    df['tokens'] = df['tokens'].apply(literal_eval)
    print("----- 전처리된 데이터 불러오기 성공 -----")
except FileNotFoundError:
    print("오류: '../data/processed_reviews.csv' 파일을 찾을 수 없습니다.")
    exit()

documents = df['tokens'].apply(lambda tokens: ' '.join(tokens))

# 2. BERTopic 실행 (오직 '주제 분류' 역할만 수행)
print("\n----- BERTopic 모델 학습 시작 (주제 분류 목적) -----")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"----- 사용할 디바이스: {device.upper()} -----")

embedding_model = SentenceTransformer("jhgan/ko-sbert-nli", device=device)

# 키워드 추출은 직접 할 것이므로, Vectorizer는 사용하지 않습니다.
topic_model = BERTopic(embedding_model=embedding_model,
                       verbose=True,
                       min_topic_size=3)

topics, probs = topic_model.fit_transform(documents)
df['topic'] = topics
print("----- 주제 분류 완료 -----")


# 3. [논문 핵심] '올바른' 키워드 직접 계산하기
# BERTopic의 키워드 추출 엔진 대신, 우리가 직접 각 토픽의 핵심 키워드를 계산합니다.
print("\n----- '올바른' 키워드 직접 계산 시작 -----")

topic_keywords = {}
for topic_num in df['topic'].unique():
    if topic_num == -1: continue # -1 토픽(이상치)은 제외
    
    # 해당 토픽에 속하는 모든 리뷰의 단어들을 하나의 리스트로 합칩니다.
    all_tokens_in_topic = sum(df[df['topic'] == topic_num]['tokens'].tolist(), [])
    
    # 단어 빈도수를 계산하여 가장 많이 등장한 상위 5개 단어를 키워드로 저장합니다.
    word_counts = Counter(all_tokens_in_topic)
    top_keywords = [word for word, count in word_counts.most_common(5)]
    topic_keywords[topic_num] = ', '.join(top_keywords)

print("✅ '올바른' 키워드 계산 완료:")
print(topic_keywords)

# [핵심] BERTopic 모델 객체에 우리가 직접 계산한 키워드를 강제로 설정(업데이트)합니다.
custom_labels = {topic: keywords for topic, keywords in topic_keywords.items()}
topic_model.set_topic_labels(custom_labels)


# 4. 결과 저장
# 다음 분석 단계에서 사용할 수 있도록 결과물들을 저장합니다.
if not os.path.exists("../models"): os.makedirs("../models")
if not os.path.exists("../data"): os.makedirs("../data")

# 각 리뷰에 토픽 번호가 할당된 데이터프레임을 저장합니다.
df.to_csv('../data/reviews_with_topics.csv', index=False, encoding='utf-8-sig')

# 직접 계산한 키워드가 업데이트된 토픽 모델을 저장합니다.
topic_model.save("../models/bertopic_model_final", serialization="safetensors")

print("\n\n✅ 모든 작업이 완료되었습니다.")
print("결과물:")
print("1. '../data/reviews_with_topics.csv' (리뷰별 토픽 번호)")
print("2. '../models/bertopic_model_final' (키워드가 수정된 토픽 모델)")


----- 전처리된 데이터 불러오기 성공 -----

----- BERTopic 모델 학습 시작 (주제 분류 목적) -----
----- 사용할 디바이스: CPU -----


2025-09-25 22:09:37,173 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2025-09-25 22:09:39,748 - BERTopic - Embedding - Completed ✓
2025-09-25 22:09:39,750 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-25 22:09:47,685 - BERTopic - Dimensionality - Completed ✓
2025-09-25 22:09:47,686 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-25 22:09:47,693 - BERTopic - Cluster - Completed ✓
2025-09-25 22:09:47,696 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-09-25 22:09:47,711 - BERTopic - Representation - Completed ✓


----- 주제 분류 완료 -----

----- '올바른' 키워드 직접 계산 시작 -----
✅ '올바른' 키워드 계산 완료:
{np.int64(5): '범계, 술집, 꼬치, 모찌리도후, 맛집', np.int64(0): '구이, 일본, 볶음밥, 범계, 웨이팅', np.int64(3): '범계, 술집, 안주, 분위기, 맛집', np.int64(1): '이자카야, 범계, 이태원, 술집, 사장', np.int64(7): '친구, 안주, 범계, 술집, 최고', np.int64(2): '친절, 직원, 음식, 감사, 최고', np.int64(4): '안주, 오늘, 분위기, 만족, 저녁', np.int64(6): '친구, 웨이팅, 대기, 시간, 남자'}


✅ 모든 작업이 완료되었습니다.
결과물:
1. '../data/reviews_with_topics.csv' (리뷰별 토픽 번호)
2. '../models/bertopic_model_final' (키워드가 수정된 토픽 모델)
